In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class SimpleRNN(nn.Module):
    def __init__(self, input_size=9, hidden_size=4, output_size=3):
        super(SimpleRNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = nn.Linear(input_size, hidden_size)    # 9x4
        self.hidden_to_hidden = nn.Linear(hidden_size, hidden_size)  # 4x4
        self.hidden_to_output = nn.Linear(hidden_size, output_size)  # 4x3


    def forward(self, inputs): # (10,9)
        steps_output, hidden_states = {}, {}

        hidden_states[-1] = torch.zeros((1, self.hidden_size))  # (1,4)


        for t in range(len(inputs)):
            x = inputs[t].reshape(1,9)    # (1,9)

            hidden_cur = self.input_to_hidden(x)  # (1,9) * (9,4) = (1,4)
            h_prev = self.hidden_to_hidden(hidden_states[t - 1])  # (1,4) * (4,4) = (1,4)
            hidden_states[t] = torch.tanh(hidden_cur + h_prev)  # (1,4) + (1,4) = (1,4)

            y_t = self.hidden_to_output(hidden_states[t])  # (1,4) * (4,3) = (1,3)

            steps_output[t] = y_t

        return steps_output, hidden_states


if __name__ == '__main__':
    sequence_length = 10
    input_size = 9
    hidden_size = 4
    output_size = 3

    model = SimpleRNN(input_size, hidden_size, output_size)

    inputs = [torch.randn(input_size) for _ in range(sequence_length)]

    output, hidden_states = model(inputs)
    print("\nFinal output keys:", output.keys())


Final output keys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [8]:
sequence_length = 10
input_size = 9
hidden_size = 4
output_size = 3

model = SimpleRNN(input_size, hidden_size, output_size)

inputs = [torch.randn(input_size) for _ in range(sequence_length)] # 10 sequences, each 9 features

output, hidden_states = model(inputs)
print("\nFinal output keys:", output.keys())


Final output keys: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
